<a href="https://colab.research.google.com/github/zeon2103133/applied_data_science_capstone_project/blob/main/Copy_of_MVP_Colab_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 25.5 MB/s eta 0:00:00


In [ ]:
!playwright install

154.7 MiB [] 0% 10.2s154.7 MiB [] 0% 9.3s154.7 MiB [] 0% 6.8s154.7 MiB [] 0% 5.0s154.7 MiB [] 1% 4.4s154.7 MiB [] 1% 4.1s154.7 MiB [] 2% 4.0s154.7 MiB [] 2% 4.2s154.7 MiB [] 2% 4.3s154.7 MiB [] 3% 4.4s154.7 MiB [] 3% 4.0s154.7 MiB [] 4% 4.1s154.7 MiB [] 4% 4.3s154.7 MiB [] 5% 4.0s154.7 MiB [] 5% 3.9s154.7 MiB [] 6% 4.0s154.7 MiB [] 6% 3.9s154.7 MiB [] 7% 3.8s154.7 MiB [] 7% 3.9s154.7 MiB [] 7% 3.7s154.7 MiB [] 8% 3.5s154.7 MiB [] 9% 3.5s154.7 MiB [] 10% 3.5s154.7 MiB [] 11% 3.4s154.7 MiB [] 11% 3.3s154.7 MiB [] 12% 3.3s154.7 MiB [] 12% 3.2s154.7 MiB [] 13% 3.1s154.7 MiB [] 13% 3.2s154.7 MiB [] 14% 3.1s154.7 MiB [] 15% 3.1s154.7 MiB [] 15% 3.0s154.7 MiB [] 16% 3.0s154.7 MiB [] 17% 3.0s154.7 MiB [] 17% 2.9s154.7 MiB [] 18% 2.8s154.7 MiB [] 19% 2.7s154.7 MiB [] 20% 2.6s154.7 MiB [] 21% 2.5s154.7 MiB [] 22% 2.4s154.7 MiB [] 23% 2.4s154.7 MiB [] 24% 2.3s154.7 MiB [] 25% 2.2s154.7 MiB [] 26% 2.2s154.7 MiB [] 26% 2.3s154.7 MiB [] 27% 2.2s154.7 MiB [] 28% 2.2s154.7 MiB [] 29% 2.1s154.7 MiB [] 

In [ ]:
from playwright.async_api import async_playwright

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape table data from a page
async def scrape_table_data(page):
    # Extracting HTML content from the page
    html = await page.inner_html("body")
    # Parsing HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    # Selecting table rows from the HTML content
    rows = soup.select("tbody.Crom_body__UYOcU tr")
    # Extracting data from each row and storing in a list of lists
    data = [[td.get_text() for td in row.find_all("td")] for row in rows]
    return data

# Main function to execute the scraping process
async def main():

    # Setting up Playwright
    async with async_playwright() as p:

        # Launching a Chromium browser
        browser = await p.firefox.launch(headless=True)
        # Creating a new browsing context
        context = await browser.new_context()
        # Creating a new page within the context
        page = await context.new_page()



        # Navigating to the NBA stats website
        await page.goto("https://www.nba.com/stats/players/advanced", timeout=60000)

        # List to store scraped data
        data = []
        # Variable to track page number
        page_number = 1

        # Looping through pages until there are no more next buttons
        while True:
            try:
                # Scraping data from the current page
                data += await scrape_table_data(page)
                print(f"Scraped data from page {page_number}.")

                # Finding the next button
                next_button = await page.query_selector("button[data-track='click'][data-type='controls'][data-pos='next']")
                # If no next button found, exit loop
                if not next_button:
                    print("No next button found. Exiting loop.")
                    break

                # Clicking the next button
                await next_button.click(timeout=5000)
                # Waiting for table data to load on the next page
                await page.wait_for_selector("tbody.Crom_body__UYOcU tr")
                # Incrementing page number
                page_number += 1
            except TimeoutError as e:
                # Handling timeout error
                print(f"Failed to navigate to the next page: {e}")
                print("Saving the DataFrame and exiting.")
                break
            except Exception as e:
                # Handling other exceptions
                print(f"An error occurred: {e}")
                print(f"error during reading page: {page_number}")
                break

        # Column names for the DataFrame
        columns = [
            "Player", "Team", "Opponent", "Date", "Result", "Minutes", "Points",
            "FGM", "FGA", "FG%", "3PM", "3PA", "3P%", "FTM", "FTA", "FT%",
            "OREB", "DREB", "REB", "AST", "TO", "STL", "BLK", "PF", "+/-", "SPI"
        ]
        # Creating a DataFrame from the scraped data
        df = pd.DataFrame(data)
        print(df.head())
        print(df.shape)
        # Saving DataFrame to a CSV file
        #df.to_csv("nba_player_boxscores_2023-2024.csv", index=False)
        print("Scraping completed.")
        return df

# Call the main function to start the scraping process
df = await main()

Scraped data from page 1.
Scraped data from page 2.
Scraped data from page 3.
Scraped data from page 4.
Scraped data from page 5.
Scraped data from page 6.
Scraped data from page 7.
Scraped data from page 8.
Scraped data from page 9.
Scraped data from page 10.
Scraped data from page 11.
Scraped data from page 12.
An error occurred: Timeout 5000ms exceeded.
error during reading page: 12
  0                1    2   3   4   5   6     7      8      9   ...    14  \
0  1      Buddy Hield  PHI  31  76  38  38  26.3  117.3  117.2  ...   2.6   
1  2       Obi Toppin  IND  26  75  42  33  21.3  119.4  117.3  ...   4.8   
2  3   Andre Drummond  CHI  30  74  35  39  17.0  114.5  113.7  ...  18.6   
3  3    Austin Reaves  LAL  25  74  41  33  32.1  114.7  115.9  ...   2.2   
4  3  Christian Braun  DEN  22  74  51  23  19.4  108.6  110.5  ...   4.5   

     15    16    17    18    19    20      21    22    23  
0   9.8   6.1   8.3  57.0  58.1  18.4  101.60   8.3  4235  
1  13.3   9.0   8.5  65.7  6

In [ ]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
560,554,Kaiser Gates,NOP,27,1,0,1,7.4,81.3,88.2,...,0.0,9.1,4.5,0.0,0.0,0.0,22.2,106.55,-15.0,16
561,554,Malcolm Cazalon,DET,22,1,0,1,2.6,183.3,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.48,0.0,6
562,554,Markquis Nowell,TOR,24,1,0,1,3.5,128.6,62.5,...,50.0,0.0,28.6,0.0,0.0,53.2,22.2,102.37,35.7,7
563,554,Quenton Jackson,IND,25,1,1,0,1.8,0.0,120.0,...,0.0,50.0,25.0,0.0,0.0,0.0,0.0,132.11,18.2,5
564,554,Ron Harper Jr.,TOR,23,1,1,0,3.7,100.0,188.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.78,-4.5,8


In [ ]:
df_7to23 = df.drop(columns=range(7))

In [ ]:
df_7to23.corr()

<ipython-input-15-e37b4dcb8b5d>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_7to23.corr()


""
